In [1]:
import pandas as pd 
import numpy as np
import networkx as nx
import csv
from collections import defaultdict, deque
import sys
from matplotlib import pyplot as plt
import math
import pickle
import time

n = 250
m = 250

In [ ]:
sources = defaultdict(int)
sinks = defaultdict(int)
follows = defaultdict(set)
followedby = defaultdict(set)

In [ ]:
f = open('../train.txt')
lines = 0
for line in f.readlines():
    splitted_line = line.split()
    src, dests = splitted_line[0], splitted_line[1:]
    src = int(src)
    dests = [int(x) for x in dests]
    
    sources[src] = len(dests)
    follows[src].update(dests)
    
    for dest in dests:
        sinks[dest] += 1
        followedby[dest].add(src)
    print(lines, len(dests), len(set(dests)))
    lines+=1
    
f.close()

In [ ]:
print('dict(int) sizes in Mb: ', sys.getsizeof(sources) / 10**6, sys.getsizeof(sinks) / 10**6)
print('dict(set) sizes in Mb: ', sys.getsizeof(follows) / 10**6, sys.getsizeof(followedby) / 10**6)
print('number of sources and sinks: ', len(sources), len(sinks))

## 1. select n super sources and m sinks

In [ ]:
# super sources (follows many perple)
plt.plot(sorted(list(sources.values()))[20000-n:])
print('super sources with >=', sorted(list(sources.values()))[20000-n], 'links')

In [ ]:
# super sinks (followed by many people)
plt.plot(sorted(list(sinks.values()))[4867136-m:])
print('super sinks with >=', sorted(list(sinks.values()))[4867136-m], 'links')

In [ ]:
super_sources = list(map(lambda x: int(x[0]) ,sorted(sources.items(), key=lambda item: item[1], reverse = True))) #[:n]
super_sinks = list(map(lambda x: int(x[0]) ,sorted(sinks.items(), key=lambda item: item[1], reverse = True))) # [:m]

## 2. build groups from super nodes

In [ ]:
groups = [set() for i in range(m+n)]

In [ ]:
def depthLimitedSearch(adjlist, visited, depth, limit, source, supernodes):
    visited.add(source)
    if depth < limit:
        #if len(adjlist[source]) > 0:
        #    print(depth, len(adjlist[source]))
        for follow in adjlist[source]:
            if follow not in supernodes:
                depthLimitedSearch(adjlist, visited, depth+1, limit, follow, supernodes)
    return

In [ ]:
# run bfs from all super nodes, depth limited set to 2
limit = 2
group_id = 0
total = 0
for source in super_sources[:n]:
    depthLimitedSearch(follows, groups[group_id], 0, limit, source, set(super_sources[:2000]))
    print(group_id, len(groups[group_id]))
    total += len(groups[group_id])
    group_id += 1

print(total)

total = 0
for sink in super_sinks[:m]:
    depthLimitedSearch(followedby, groups[group_id], 0, limit, sink, set(super_sinks[:1000]))
    print(group_id, len(groups[group_id]))
    total += len(groups[group_id])
    group_id += 1

print(total)

### dump

In [ ]:
with open('../super_sources.pickle', 'wb') as handle:
    pickle.dump(super_sources, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../super_sinks.pickle', 'wb') as handle:
    pickle.dump(super_sinks, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('../groups.pickle', 'wb') as handle:
    pickle.dump(groups, handle, protocol=pickle.HIGHEST_PROTOCOL)

### load

In [ ]:
groups = [set() for i in range(m+n)]

In [10]:
with open('../super_sources.pickle', 'rb') as handle:
    super_sources = pickle.load(handle)

with open('../super_sinks.pickle', 'rb') as handle:
    super_sinks = pickle.load(handle)

In [3]:
with open('../groups.pickle', 'rb') as handle:
    groups = pickle.load(handle)

## 3. generate features for edges

In [4]:
list_of_features = []

In [5]:
list_of_features = []
a = time.perf_counter()
f = open('../train.txt')
#fout = open("../features.txt", "w")

row = 0
for line in f.readlines():
    if row > 1000000:
        break
    splitted_line = line.split()
    src, dests = splitted_line[0], splitted_line[1:]
    src = int(src)
    dests = [int(x) for x in dests]
    src_in = [1 if src in group else 0 for group in groups]
    for d in dests:
        d_in = [1 if d in group else 0 for group in groups]
        list_of_features.append( np.int8([src_in[i] + d_in[i]//2 for i in range(m+n)]))
        if row % 100000 == 0:
            print(list_of_features[row])
            print(row, 'group(set) sizes in Mb: ', sys.getsizeof(list_of_features) / 10**6) 
            b = time.perf_counter()
            print(b-a)
        row += 1

f.close() 
while row > 0:
    list_of_features.append( np.int8([0 for i in range(m+n)]))
    if row % 100000 == 0:
        print(list_of_features[row])
        print(row, 'group(set) sizes in Mb: ', sys.getsizeof(list_of_features) / 10**6) 
        b = time.perf_counter()
        print(b-a)
    row -= 1
print('Done')    
#fout.close()

[1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 0 1 1 1
 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1
 1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 0 0
 1 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 0 0 1
 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0
 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1
 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1 1 1
 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 0 0
 0 1 1 0 1 1 1 1 1 1 0 1 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [6]:
super_sources = 0
super_sinks = 0
groups = 0

In [7]:
len(list_of_features)

2045072

In [8]:
list_of_features = np.asarray(list_of_features)
np.savetxt("../features.csv", list_of_features, delimiter=",")
f.close()

KeyboardInterrupt: 

In [11]:
df = pd.DataFrame(list_of_features, columns=super_sources[:n]+super_sinks[:m])

In [12]:
df

,3361377,3509903,3280389,1580988,3638272,2827436,427805,3438576,805360,2981537,...,1618616,3082645,4103612,4183784,2650412,1510806,1919568,4313572,973393,526798
0,1,1,1,1,1,1,1,1,1,0,...,1,0,1,1,0,1,0,0,1,1
1,1,1,1,1,1,1,1,1,1,0,...,1,0,1,1,0,1,0,0,1,1
2,1,1,1,1,1,1,1,1,1,0,...,1,0,1,1,0,1,0,0,1,1
3,1,1,1,1,1,1,1,1,1,0,...,1,0,1,1,0,1,0,0,1,1
4,1,1,1,1,1,1,1,1,1,0,...,1,0,1,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045067,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2045068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2045069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2045070,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.to_csv('../features.csv')

## 4. combine features

## 5. Train and test models

In [15]:
y = pd.DataFrame(np.concatenate((np.ones(1022536, np.int8), np.zeros(1022536, np.int8)),axis=0), columns = ['exist'])

In [19]:
from sklearn.linear_model import LogisticRegression, SGDRegressor

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    y, test_size=0.30, 
                                                    random_state=101)


# Fit the model on training set
model = LogisticRegression(solver = 'lbfgs')
# model = SGDRegressor()
model.fit(X_train, y_train.values.ravel())
# save the model to disk
filename = '../model_harry.sav'
pickle.dump(model, open(filename, 'wb'))





In [18]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9825271139421243


In [20]:
list_of_features = 0
df = 0

In [ ]:
result = pd.DataFrame()
result['Id'] = test_raw['ID']
result['Predicted'] = pred_data
result.to_csv('MultinomialNB.csv', sep=',', index=False)

## 6. submit prediction